In [25]:
import tkinter as tk
from tkinter import filedialog

In [26]:
root = tk.Tk()
root.withdraw()

ld_path = filedialog.askopenfilename()
ld_path

'C:/Users/lci734/OneDrive - AFRY/Documents/01_poyry/01_projetos/02_ternium/2023 07 04 - LDD - FUNDAÇÃO REGENERADORES - REV01.xlsx'

### Verificar se o registro já existe (com base no projeto e na data de status da LDD)
### Caso já exista, perguntar se deseja substituir valores (se sim, apagar existente e inserir novo)
### Caso não exista, apenas inserir valores

In [27]:
import pandas as pd

In [28]:
df =  pd.read_excel(ld_path, sheet_name='GERAL Unificadas')
ldd_start = df.loc[df['Unnamed: 1'] == 'PROJETO'].index[0] + 1
df = df.iloc[ldd_start: , :34]
df.columns = ['bcodigo_id','projeto','n_ficha','disciplina','n_poyry',
              'n_cliente','area','nome_area','fase_projeto','titulo',
              'inicio_bl','termino_bl','inicio_rp','termino_rp',
              'revisao_inicial','data_emissao_inicial','revisao_ultima',
              'data_emissao_ultima','status_emissao','grd','folhas_previstas',
              'folhas_executadas','formato','a1eq_bl','a1eq_rp','avanco',
              'a1eq_real','status_comentario','revisao_retorno','data_retorno',
              'status_retorno','status_cliente','obs','marco']
df = df[['projeto', 'disciplina', 'n_poyry', 'n_cliente', 'avanco', 'a1eq_bl', 'a1eq_rp', ]].reset_index(drop = True)



df.head()

,projeto,disciplina,n_poyry,n_cliente,avanco,a1eq_bl,a1eq_rp
0,109003106-002,CONCRETO,AF01-E-0602,CS.AF01.REGS.GRAL-HME10040_0A,1,11.25,1.75
1,109003106-002,CONCRETO,AF01-E-0601,CS.AF01.REGS.GRAL-HRL10041_0A,1,1.625,1.625
2,109003106-002,CONCRETO,AF01-Z-0600,CS.AF01.REGS.GRAL-HCA10031_0A,1,2.5,5.375
3,109003106-002,CONCRETO,AF01-C04-0001,CS.AF01.REGS.GRAL-HFO10001_0A,1,1,1
4,109003106-002,CONCRETO,AF01-C04-0002,CS.AF01.REGS.GRAL-HFU10021_0A,1,1,1


In [29]:
projeto = df.projeto[0]

In [30]:
from datetime import datetime

def validar_data(prompt):
    while True:
        data_input = input(prompt)
        try:
            data_status = datetime.strptime(data_input, "%d/%m/%Y").date()
            return data_status
        except ValueError:
            print("Data inválida. Por favor, digite uma data no formato DD/MM/YYYY.")

data_status = validar_data("Digite a data de status (DD/MM/YYYY): ")
print("Data validada:", data_status)

Digite a data de status (DD/MM/YYYY): 04/07/2023
Data validada: 2023-07-04


In [31]:
tipo = 'LISTA DE DESENHOS E DOCUMENTOS'

In [32]:
import sqlite3 as sql
import numpy as np

In [33]:
con = sql.connect('teste_00.db')
cur = con.cursor()

In [34]:
# Pegando id_disciplina
cur.execute("SELECT * FROM disciplinas")
disciplinas = cur.fetchall()
disciplinas = pd.DataFrame(disciplinas)
disciplinas.columns = ['id_disciplina', 'disciplina']
disciplinas.head()

df = df.merge(disciplinas[['disciplina', 'id_disciplina']], 
                           left_on='disciplina', 
                           right_on='disciplina',
                           how='left')

df.id_disciplina = df.id_disciplina.astype('object')

In [35]:
# Inserindo projeto na tabela projetos
insert_query = "INSERT OR IGNORE INTO projetos (n_projeto) VALUES (?)"
cur.execute(insert_query, (projeto,))

query = "SELECT * FROM projetos WHERE n_projeto = ?"
cur.execute(query, (projeto,))
id_projeto = cur.fetchall()[0][1]

# Inserindo registro na tabela registros
check_query = "SELECT COUNT(*) FROM registros WHERE id_projeto = ? AND data_status = ? AND tipo = ?"
cur.execute(check_query, (id_projeto, data_status, tipo,))
unico = cur.fetchall()[0][0]

if unico == 0:
    insert_query = "INSERT OR IGNORE INTO registros (id_projeto, data_status, tipo) VALUES (?, ?, ?)"
    cur.execute(insert_query, (id_projeto, data_status, tipo,))
    print('Registro inserido')
else:
    print('Registro já existente.')

# Inserindo LDD na tabela documentos
query = "SELECT * FROM registros WHERE id_projeto = ? AND data_status = ? AND tipo = ?"
cur.execute(query, (id_projeto, data_status, tipo,))
id_registro = cur.fetchall()[0][0]

check_query = "SELECT COUNT(*) FROM documentos WHERE id_registro = ?"
cur.execute(check_query, (id_registro,))
unico = cur.fetchall()[0][0]

if unico == 0:
    n_poyry = df.n_poyry.values
    n_cliente = df.n_cliente.values
    avanco = df.avanco.values
    a1eq_bl = df.a1eq_bl.values
    a1eq_rp = df.a1eq_rp.values
    id_disciplina = df.id_disciplina.values
    id_registro = [id_registro] * len(n_cliente)    
    
    insert_query = """INSERT INTO documentos 
    (
    n_poyry,
    n_cliente,
    avanco,
    a1eq_bl,
    a1eq_rp,
    id_disciplina,
    id_registro
    )
    VALUES
    (?, ?, ?, ?, ?, ?, ?)"""
    cur.executemany(insert_query, list(zip(n_poyry, n_cliente, avanco, a1eq_bl, a1eq_rp, id_disciplina, id_registro,)))
    print('LDD inserida')
else:
    print('LDD já existente.')

Registro inserido
LDD inserida


In [36]:
con.commit()
con.close()

#### Recuperar valores da tabela "disciplinas" e respectivo id_disciplina
#### Merge do id_disciplina com tabela documentos

#### Recuperar Nº do Projeto a partir da LDD
#### Inserir projeto na tabela "projetos"

#### Recuperar id_projeto

#### Inserir na tabela "registros"

#### Recuperar id_registro

#### Inserir na tabela "documentos"